In [ ]:
# -q (Quiet Mode) :
# Useful when you don’t want to see a lot of logs, especially in scripts or notebooks.

# -U (Upgrade Mode) : 
# Ensures that the latest version of the package is installed. 
# If an older version is already installed, it will upgrade to the latest available version.

! pip install -qU langchain-pinecone langchain-google-genai

In [ ]:

import os
from dotenv import load_dotenv  # helps to load env files


load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
# print("GOOGLE_API_KEY : : : ", GOOGLE_API_KEY)


In [ ]:
from pinecone import Pinecone , ServerlessSpec

# Create PineCone Object 
pc = Pinecone(api_key=PINECONE_API_KEY )

pc

### Creating PineCone Index 

In [ ]:
import time

index_name = "langchain-test-index"  # change if desired

# First delete previous one then re-initialise index on pinecone where our data will goona save 
pc.delete_index(index_name)  # Deletes the existing index
time.sleep(10)  # Wait for the deletion to complete
print("Index Recreate Successfully")
print(f"Connected to existing index: {index_name}")

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        # ServerlessSpec allows to make a region to store data on a given index 
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


index = pc.Index(index_name)
index , pc.index_api

### Create Embedding 
#### Use Google Embedding Technique 

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding  = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embedding


In [ ]:
vector = embedding.embed_query("We are building a rag system")

vector

### Add PineCone vector store to store created embedding on a particular index 

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index , embedding=embedding)

vector_store

### Data Save  with some dummy document

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)


document_1

In [ ]:
# Library to generate unique ids
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
len(documents)

In [ ]:
# Documents based embedding stores on unique indexes

uuids = [str(uuid4()) for _ in range(len(documents))]

uuids

vector_store.add_documents(documents=documents, ids=uuids)

### Data Reterival 

In [ ]:
result = vector_store.similarity_search(
    "Langchain provides abstractions to make working with LLMs easy?",
    k = 2,
    filter={"source": "tweet"}
)

for res in result:
    print(f"* {res.page_content} [{res.metadata}]")


### Similarity search with score We can also search with score

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?"
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

### Creating Model

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize model
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [ ]:
def answer_to_user(query: str):
    search_results = vector_store.similarity_search(query)
    
    retrieved_texts = "\n".join([doc.page_content for doc in search_results])

    prompt = f"Use the following information to answer the query:\n\n{retrieved_texts}\n\nQuery: {query}"

    final_answer = model.invoke(prompt)
    return final_answer

In [ ]:
# Example usage
query = "Tell me about Langchain?"
response = answer_to_user(query)

print(response.content)